# Lab 2. Solar Panel Instructions

## Introduction

In this notebook we show you how to create your second sub-agent on Amazon Bedrock Agents.

This agent contains instructions on how to install and to do maintenance on Solar Panels, where customers can ask the agent to return these information from an [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/).

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 

Amazon Bedrock Knowledge Bases is Bedrock's fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.

In the context of our agent, if the answer to a question is not present on the Knowledge Base, customers can ask agent to create a support ticket, to get a human in the loop to help with their questions.

The following represents the piece of architecture that will be built on this module.

![Architecture](img/solar_panel_agent.png)

## Setup

Make sure that your boto3 version is the latest

If not, return to [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run the Setup block again.

In [1]:
!pip freeze | grep boto3

boto3==1.40.55


Get your workshop ID / resource suffix. If not found, return to [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run the Setup block again.

In [2]:
import os

def get_workshop_id():
    workshop_id_file = '../.workshop_id'
    if os.path.exists(workshop_id_file):
        with open(workshop_id_file, 'r') as f:
            return f.read().strip()
    else:
        return None
    
workshop_id = get_workshop_id()
resource_suffix = f"{workshop_id}"

if workshop_id is None:
    print("No workshop ID found. Please run the Setup script in notebook 1.")
else:
    print("Your resource suffix is", resource_suffix)

Your resource suffix is 7eb12cba


## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create out second agent.

In [3]:
import boto3
import os

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [4]:
solar_agent_name = f"solar-p-{resource_suffix}"

solar_lambda_name = f"fn-solar-p-{resource_suffix}"

solar_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solar_agent_name}'

dynamodb_table = f"{solar_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{solar_agent_name}-kb'

knowledge_base_description = "KB containing solar panel instructions for installation and maintenance"
bucket_name = f'solar-p-kb-{account_id}-{resource_suffix}'

### Importing helper functions

In the following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

In `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

In `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [5]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Load Knowledge Base

In this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

In the next steps we will generate the data used to populate the knowledge base. It will be composed of instructions on how to handle a solar panel

**This creation process takes several minutes.**

In [6]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB solar-p-7eb12cba-kb
Step 1 - Creating or retrieving solar-p-kb-833005555478-7eb12cba S3 bucket for Knowledge Base documents
Creating bucket solar-p-kb-833005555478-7eb12cba
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_774) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '315',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Sat, 18 Oct 2025 14:28:24 '
                                                 'GMT',
                                         'x-amzn-requestid': '43d0de2e-513b-408e-9631-cc2bbca67a42'},
                        'HTTPStatusCode': 200,
                        'RequestId': '43d0de2e-513

### Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [7]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

The kb_documents directory already exists!


Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [8]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

Generating one file with instructions on how to install a solar panel.

In [9]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step on how to install a solar panel at home.
    You know the following fictional solar panel models: Sunpower X, Sunpower Y
    and Sunpower double-X. For each one of those models, provide some general
    model description and its features. Next provide a numbered list describing
    how to install each model of solar panel. Include information about how to
    ensure compliance with energy rules.

    Answer only with the instructions and solar panel descriptions.
    Avoid answer with afirmations like: "OK, I can generate it,",
    "As an expert on clean energy, I ", or "Yes, please find following example."
    Be direct and only reply the instructions and descriptions.
'''

solar_energy_file_name = 'solar-panel-instructions.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)
description_and_instructions = response_message['content'][0]['text']
print(f"Generated data:\n{description_and_instructions}")
write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    description_and_instructions
)

Generated data:
Sunpower X:
The Sunpower X is a high-efficiency monocrystalline solar panel designed for residential and commercial installations. It features a sleek black-on-black design and is known for its exceptional performance in low-light conditions. The Sunpower X has a maximum power output of 345 watts and a module efficiency of up to 22.8%.

Sunpower Y:
The Sunpower Y is a versatile polycrystalline solar panel suitable for a wide range of applications, including ground-mounted and rooftop systems. It offers a balance between efficiency and affordability, with a maximum power output of 320 watts and a module efficiency of up to 19.5%. The Sunpower Y is known for its durability and resistance to harsh environmental conditions.

Sunpower double-X:
The Sunpower double-X is a premium monocrystalline solar panel designed for maximum energy production. It features a unique double-panel design that captures sunlight from both the front and rear sides, increasing overall energy yield

### Generating data prompt
Generating another file with instructions on how to do maintenance on a solar panel.

In [10]:
text_generation_energy_instructions = f'''
    You will be act as an expert on clean energy.
    You know the following fictional solar panel models: Sunpower X, Sunpower Y
    and Sunpower double-X. Here is are some descriptions of the different
    models and how to install them:
    <description_and_instructions>
    {description_and_instructions}
    </description_and_instructions>
    Generate a step-by-step instructions on how to do maintenance on each of
    those models at a regular home. Include information about how to
    ensure consistent compliance with energy rules.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-maintenance.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)
print(f"Generated data:\n{response_message['content'][0]['text']}")

write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    response_message['content'][0]['text']
)


Generated data:
Here are step-by-step instructions for maintaining the different Sunpower solar panel models at a regular home, including information about ensuring consistent compliance with energy rules:

Sunpower X Maintenance:

1. Inspect the solar panels regularly for any visible damage, such as cracks, discoloration, or delamination.
2. Clean the solar panels periodically to remove dirt, dust, and debris that can reduce their efficiency. Use a soft brush or cloth and a mild, non-abrasive cleaning solution.
3. Check the mounting system and hardware for any signs of corrosion or loosening, and tighten or replace components as needed.
4. Monitor the system's performance and compare it to the expected output. If there is a significant drop in energy production, contact a professional for troubleshooting and maintenance.
5. Ensure compliance with local energy regulations by obtaining necessary permits, following installation guidelines, and adhering to safety codes.
6. Keep records of

### Uploading data to S3
Uploading generated files into S3 Bucket.

In [11]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [12]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_documents/solar-panel-instructions.txt to solar-p-kb-833005555478-7eb12cba
uploading file kb_documents/solar-panel-maintenance.txt to solar-p-kb-833005555478-7eb12cba
{ 'dataSourceId': 'FMOPV2MXLM',
  'ingestionJobId': 'AZSCQOK0WL',
  'knowledgeBaseId': 'CPDC3FMTP1',
  'startedAt': datetime.datetime(2025, 10, 18, 14, 32, 21, 646170, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 10, 18, 14, 32, 21, 646170, tzinfo=tzlocal())}
{ 'dataSourceId': 'FMOPV2MXLM',
  'ingestionJobId': 'AZSCQOK0WL',
  'knowledgeBaseId': 'CPDC3FMTP1',
  'startedAt': datetime.datetime(2025, 10, 18,

## Creating Agent

Create the Solar Panel agent that will have a Knowledge Base and also a Lambda action group to handle exception workflow (when information is not found on KB).

For this agent we will use the following instructions:
```
You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Providing maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner
5. NEVER invent information not available in your knowledge base

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user
```
We will also connect a knowledge base with the information about solar panels

```
Access the knowledge base when customers ask about to install and maintain solar panels
```
And we will make the following tool available to the agent:
- `open_ticket`: to open new support tickets
- `get_ticket_status`: to get the current status of an existing ticket

In [13]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [14]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about to install and maintain solar panels'
}

In [15]:
agent_instruction = """You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Offering maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user"""

agent_description = """You are a solar energy helper bot. 
    You can retrieve information on how to install and do maintenance on solar panels"""

solar_agent = agents.create_agent(
    solar_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)
solar_agent

('QVU1TYDKDP',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:833005555478:agent-alias/QVU1TYDKDP/TSTALIASID')

In [16]:
agents.associate_kb_with_agent(
    solar_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

Waiting for agent status to change. Current status CREATING
Agent id QVU1TYDKDP current status: NOT_PREPARED


### Creating Action Group

On this session, we're going create an action group to handle support tickets and associate it with our agent. To do so, we will first create a Lambda function code to fulfill the execution of the agent's actions Next we will define the actions available actions that an agent can take using function details. Similar to the previous agent, you can also define the actions available using OpenAPI Schema.

#### Creating Lambda function
First let's create the lambda function

In [17]:
%%writefile solar_energy.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(customer_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': customer_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return "Thanks for contact customer {}! Your support case was generated with ID: {}".format(
        customer_id, ticket_id
    )

def get_ticket_status(customer_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         customer_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(customer_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(customer_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Overwriting solar_energy.py


### Defining available actions
Now it's time to define the actions that can be taken by the agent

In [18]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related with solar panel or clean energy""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "msg": {
                "description": "The reason why customer is opening a ticket",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_ticket_status",
        "description": """get the status of an existing ticket""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "ticket_id": {
                "description": "Unique ticket identifier",
                "required": False,
                "type": "string"
            }
        }
    }
]

#### Associating action group to agent
Finally, we can associate a new action group with our previously created agent

In [19]:
resp = agents.add_action_group_with_lambda(
    agent_name=solar_agent_name,
    lambda_function_name=solar_lambda_name,
    source_code_file="solar_energy.py",
    agent_functions=functions_def,
    agent_action_group_name="solar_energy_actions",
    agent_action_group_description="Function to open an energy ticket for a user or get status from an opened ticket",
    dynamo_args=dynamoDB_args
)

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing maintainance question
First let's ask a question related to maintaining an existing solar panel

In [20]:
%%time
response = agents.invoke(
    "how can I check if my Sunpower double-X solar panel eletrical consumption is compliant with energy rules?", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 4d288a51-c235-4654-96f4-52eabc1ed4e7
invokeAgent API session ID: b32fe182-ac2e-11f0-b00f-16ffda0c90af
---- Step 1 ----
Took 3.5s, using 1702 tokens (in: 1512, out: 190) to complete prior action, observe, orchestrate.
To check if your SunPower double-X solar panel electrical consumption is compliant with energy rules, I will first need to search the knowledge base for relevant information on:
1. SunPower double-X solar panel specifications
2. Electrical consumption standards and regulations for solar panels
3. How to verify compliance of solar panel electrical consumption
---- Step 2 ----
Took 5.9s, using 1687 tokens (in: 1350, out: 337) to complete prior action, observe, orchestrate.
---- Step 3 ----
Took 6.0s, using 2396 tokens (in: 2112, out: 284) to complete prior action, observe, orchestrate.
Final response:

To check if your SunPower double-X solar panel electrical consumption is compliant with energy rules, you need to ensure the following:

1. Obtain 

### Testing installation question
Next let's ask a question related to installing a new solar panel

In [21]:
%%time
response = agents.invoke(
    "how can I install my Sunpower Y solar panel at home?", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 463b4ecf-dbdd-49c7-9026-959816b85422
invokeAgent API session ID: b32fe182-ac2e-11f0-b00f-16ffda0c90af
---- Step 1 ----
Took 2.9s, using 2564 tokens (in: 2414, out: 150) to complete prior action, observe, orchestrate.
To provide guidance on installing a SunPower Y solar panel at home, I will search the knowledge base for relevant information on:

1. SunPower Y solar panel specifications and requirements
2. Steps for residential solar panel installation
3. Safety guidelines and regulations to follow during installation
---- Step 2 ----
Took 3.4s, using 1621 tokens (in: 1345, out: 276) to complete prior action, observe, orchestrate.
---- Step 3 ----
Took 3.7s, using 3203 tokens (in: 2933, out: 270) to complete prior action, observe, orchestrate.
Final response:

To install SunPower Y solar panels at your home, follow these steps:

1. Evaluate your site and determine the optimal location for the solar panels, considering factors like sun exposure and potential s

### Testing personalized support
Let's now create a support ticket. To do so we will pass our customer id to trigger the support ticket creation process

In [22]:
%%time
ticket = agents.invoke(
    "Can I get support to install my Sunpower X solar panel? My customer id is 1", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(ticket)

invokeAgent API request ID: 1466593e-64f3-4976-8441-1e60d69357c8
invokeAgent API session ID: b32fe182-ac2e-11f0-b00f-16ffda0c90af
---- Step 1 ----
Took 3.1s, using 3356 tokens (in: 3225, out: 131) to complete prior action, observe, orchestrate.
The user is asking for support to install their SunPower X solar panel and has provided their customer ID. Since this seems to be a specialist-level issue that requires dedicated assistance, I should create a support ticket for them.
Using tool: open_ticket with these inputs:
[{'name': 'msg', 'type': 'string', 'value': 'Request for installation support for SunPower X solar panel'}, {'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
Thanks for contact customer 1! Your support case was generated with ID: 73f863fe-ac2f-11f0-8bff-ad1a552a1d71...

---- Step 2 ----
Took 2.7s, using 3486 tokens (in: 3453, out: 33) to complete prior action, observe, orchestrate.
Final response:
73f863fe-ac2f-11f0-8bff-ad1a552a1d71...
Agent made a 

### Testing getting support ticket details
Now let's get the details for our support ticket. For that you will need to provide the ticket id just generated in the previous query to `create_ticket`

In [23]:
%%time
response = agents.invoke(
    "Can I get details on my ticket? My customer id is 1 get my ticket id from our previous conversation {}".format(ticket), 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 9d783ced-c9b6-4455-a817-c5d24151ceb2
invokeAgent API session ID: b32fe182-ac2e-11f0-b00f-16ffda0c90af
---- Step 1 ----
Took 3.0s, using 3683 tokens (in: 3539, out: 144) to complete prior action, observe, orchestrate.
The user is asking for details on their existing support ticket. To retrieve the ticket status, I will need to call the get_ticket_status function with the customer ID and ticket ID provided.
Using tool: get_ticket_status with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}, {'name': 'ticket_id', 'type': 'string', 'value': '73f863fe-ac2f-11f0-8bff-ad1a552a1d71'}]

--tool outputs:
[{'customer_id': '1', 'description': 'Request for installation support for SunPower X solar panel', 'status': 'created', 'ticket_id': '73f863fe-ac2f-11f0-8bff-ad1a552a1d71'}]...

---- Step 2 ----
Took 2.8s, using 3901 tokens (in: 3808, out: 93) to complete prior action, observe, orchestrate.
Final response:
Based on the ticket status retrieved, yo

### Testing get open support tickets
Let's also check if our agent can get the details for more than one open case. To do so we will first create a new support case

In [24]:
%%time
response = agents.invoke(
    "Can I get support to review my Sunpower double-X solar panel consumption? My customer id is 1", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: fe9ef082-fe46-4ee4-a68b-472731f7949f
invokeAgent API session ID: b32fe182-ac2e-11f0-b00f-16ffda0c90af
---- Step 1 ----
Took 2.6s, using 4058 tokens (in: 3925, out: 133) to complete prior action, observe, orchestrate.
The user is requesting support to review the electrical consumption of their SunPower double-X solar panel. Since this seems to be a specialist-level issue requiring dedicated assistance, I should create a new support ticket for them.
Using tool: open_ticket with these inputs:
[{'name': 'msg', 'type': 'string', 'value': 'Request to review electrical consumption of SunPower double-X solar panel'}, {'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
Thanks for contact customer 1! Your support case was generated with ID: 7a424274-ac2f-11f0-8783-ad1a552a1d71...

---- Step 2 ----
Took 1.6s, using 4184 tokens (in: 4154, out: 30) to complete prior action, observe, orchestrate.
Final response:
7a424274-ac2f-11f0-8783-ad1a552a1d71..

Next we will request the overview of all open tickets

In [25]:
%%time
response = agents.invoke(
    "Can I get all tickets that I have? My customer id is 1", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 8ecb4e28-92e7-4813-baeb-229506390623
invokeAgent API session ID: b32fe182-ac2e-11f0-b00f-16ffda0c90af
---- Step 1 ----
Took 2.7s, using 4301 tokens (in: 4205, out: 96) to complete prior action, observe, orchestrate.
To retrieve all existing support tickets for the customer, I will call the get_ticket_status function with just the customer ID provided.
Using tool: get_ticket_status with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'description': 'Request for installation support for SunPower X solar panel', 'status': 'created', 'ticket_id': '73f863fe-ac2f-11f0-8bff-ad1a552a1d71'}, {'customer_id': '1', 'description': 'Request to review electrical consumption of SunPower double-X solar panel', 'status': 'c...

---- Step 2 ----
Took 3.7s, using 4651 tokens (in: 4497, out: 154) to complete prior action, observe, orchestrate.
Final response:
Based on the retrieved information, here are the open supp

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [26]:
solar_agent_alias_id, solar_agent_alias_arn = agents.create_agent_alias(
    solar_agent[0], 'v1'
)
solar_agent_id = solar_agent[0]

Store environment variables to be used on next notebooks.

In [27]:
solar_agent_arn = agents.get_agent_arn_by_name(solar_agent_name)
solar_kb = knowledge_base_name
solar_dynamodb = dynamodb_table

%store solar_agent_arn
%store solar_agent_alias_arn
%store solar_agent_alias_id
%store solar_lambda_name
%store solar_agent_name
%store solar_agent_id
%store solar_kb
%store solar_dynamodb

Stored 'solar_agent_arn' (str)
Stored 'solar_agent_alias_arn' (str)
Stored 'solar_agent_alias_id' (str)
Stored 'solar_lambda_name' (str)
Stored 'solar_agent_name' (str)
Stored 'solar_agent_id' (str)
Stored 'solar_kb' (str)
Stored 'solar_dynamodb' (str)


In [28]:
solar_agent_arn, solar_agent_alias_arn, solar_agent_alias_id

('arn:aws:bedrock:us-east-1:833005555478:agent/QVU1TYDKDP',
 'arn:aws:bedrock:us-east-1:833005555478:agent-alias/QVU1TYDKDP/20BGU5GXVZ',
 '20BGU5GXVZ')

## Next Steps
Congratulations! We've now created a solar panel agent. Next we will create our peak loader manager agent